In [7]:
import pickle
from pathlib import Path

X = pickle.load(open(Path("data") / "X.data", "rb"))
y = pickle.load(open(Path("data") / "y.data", "rb"))
# alphabet = pickle.load(open(Path("data") / "alphabet-bruno.data", "rb"))

# from sklearn.tree import DecisionTreeClassifier, plot_tree
# import matplotlib.pyplot as plt
# clf = DecisionTreeClassifier()
# clf.fit(X, y)

# plt.figure(figsize=(48,24))
# plot_tree(clf, filled=True, fontsize=10)
# plt.show()

In [27]:
# Habilita a ferramenta experimental que testa combinações de
# hiperparâmetros e exibe o desempenho deles, permitindo uma escolha melhor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

param_grid = [{"n_neighbors": [1, 3, 5, 7, 11], "weights": ["uniform", "distance"],
            "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]}]
sh = HalvingGridSearchCV(KNeighborsClassifier(), param_grid, scoring="accuracy",
                        factor=2, aggressive_elimination=False).fit(X, y)
results = pd.DataFrame(sh.cv_results_)
results[["param_" + p for p in param_grid[0]] + ["mean_test_score", "mean_train_score", "mean_score_time"]].iloc[-5:]

,param_n_neighbors,param_weights,param_algorithm,mean_test_score,mean_train_score,mean_score_time
75,1,uniform,kd_tree,0.542105,1.0,0.002341
76,1,distance,ball_tree,0.542105,1.0,0.001606
77,1,uniform,auto,0.542105,1.0,0.004580
78,1,distance,ball_tree,0.644737,1.0,0.002414
79,1,uniform,auto,0.644737,1.0,0.007879


In [10]:
from sklearn.naive_bayes import GaussianNB

param_grid = [{"var_smoothing": [1e-12, 1e-9, 1e-6]}]

sh = HalvingGridSearchCV(GaussianNB(), param_grid, scoring="accuracy",
                        factor=2, aggressive_elimination=False).fit(X, y)
                        
results = pd.DataFrame(sh.cv_results_)
results[["param_" + p for p in param_grid[0]] + ["mean_test_score", "mean_train_score", "mean_score_time"]].iloc[-5:]

,param_var_smoothing,mean_test_score,mean_train_score,mean_score_time
0,0.0,0.442105,0.740789,0.001400
1,0.0,0.452632,0.732895,0.001102
2,0.000001,0.452632,0.732895,0.001600
3,0.0,0.476316,0.671053,0.001406
4,0.000001,0.476316,0.671053,0.001400


In [30]:
from sklearn.ensemble import RandomForestClassifier

param_grid = [{"n_estimators": [50, 75, 100, 150, 200], "criterion": ["gini", "entropy", "log_loss"],
            "max_features": ["sqrt", "log2"], "class_weight": [None, "balanced"], "max_samples": [None, 0.75]}]

sh = HalvingGridSearchCV(RandomForestClassifier(), param_grid, scoring="accuracy",
                        factor=2, aggressive_elimination=True).fit(X, y)

results = pd.DataFrame(sh.cv_results_)
results[["param_" + p for p in param_grid[0]] + ["mean_test_score", "mean_train_score", "mean_score_time"]].iloc[-5:]

,param_n_estimators,param_criterion,param_max_features,param_class_weight,param_max_samples,mean_test_score,mean_train_score,mean_score_time
234,150,entropy,log2,None,None,0.542105,1.0,0.017795
235,200,entropy,sqrt,None,None,0.568421,1.0,0.019801
236,200,log_loss,sqrt,None,None,0.568421,1.0,0.020791
237,200,log_loss,sqrt,None,None,0.723684,1.0,0.021517
238,200,entropy,sqrt,None,None,0.710526,1.0,0.024380


In [37]:
from sklearn.neural_network import MLPClassifier
# Filtra warnings relacionados à não convergência do modelo, o que é esperado,
# considerando a limitação de iterações imposta para fins de teste
from warnings import filterwarnings
filterwarnings("ignore")

param_grid = [{"hidden_layer_sizes": [(100, 100, 100), (100,), (50, 50, 50, 50, 50)],
            "activation": ["logistic", "relu", "tanh"], "solver": ["adam", "lbfgs"],
            "learning_rate": ["constant", "invscaling", "adaptive"], "max_iter": [50, 100, 250]}]
            
sh = HalvingGridSearchCV(MLPClassifier(), param_grid, scoring="accuracy",
                        factor=2, aggressive_elimination=True).fit(X, y)

results = pd.DataFrame(sh.cv_results_)
results[["param_" + p for p in param_grid[0]] + ["mean_test_score", "mean_train_score", "mean_score_time"]].iloc[-5:]

,param_hidden_layer_sizes,param_activation,param_solver,param_learning_rate,param_max_iter,mean_test_score,mean_train_score,mean_score_time
322,"(100, 100, 100)",tanh,adam,adaptive,250,0.673684,0.992105,0.000800
323,"(50, 50, 50, 50, 50)",tanh,adam,constant,250,0.700000,0.986842,0.000600
324,"(100, 100, 100)",tanh,adam,invscaling,250,0.721053,0.992105,0.000802
325,"(50, 50, 50, 50, 50)",tanh,adam,constant,250,0.786842,0.946053,0.000600
326,"(100, 100, 100)",tanh,adam,invscaling,250,0.794737,0.955263,0.001000


In [41]:
from sklearn.linear_model import LogisticRegression

param_grid = [{"penalty": ["none", "l2"], "fit_intercept": [True, False],
            "class_weight": ["balanced", None], "max_iter": [50, 100, 200],
            "solver": ["newton-cg", "lbfgs", "sag", "saga"],
            "multi_class": ["ovr", "multinomial"]}]

sh = HalvingGridSearchCV(LogisticRegression(), param_grid, scoring="accuracy",
                        factor=2, aggressive_elimination=False).fit(X, y)
                        
results = pd.DataFrame(sh.cv_results_)
results[["param_" + p for p in param_grid[0]] + ["mean_test_score", "mean_train_score", "mean_score_time"]].iloc[-5:]

,param_penalty,param_fit_intercept,param_class_weight,param_max_iter,param_solver,param_multi_class,mean_test_score,mean_train_score,mean_score_time
283,none,False,None,100,newton-cg,ovr,0.873684,1.0,0.000600
284,none,False,None,50,newton-cg,ovr,0.876316,1.0,0.000601
285,none,False,balanced,50,newton-cg,ovr,0.871053,1.0,0.000200
286,none,False,balanced,200,newton-cg,ovr,0.871053,1.0,0.000601
287,none,False,balanced,100,newton-cg,ovr,0.871053,1.0,0.000200
